<br/><font size="6"><b>MegaFon Accelerator. Classification. </b></font>.<br/>

# Description of the project

## Formulation of the problem

It is necessary to build a model that predicts which of the three segments (0,1,2) each subscriber belongs to.

## Description of data

The training sample contest_train.csv consists of the following columns:

* ID - subscriber identifier.
* TARGET - segment corresponding to the subscriber.
* FEATURE_0…FEATURE_259 — subscriber characteristics.

* The test sample contest_test.csv consists of the ID column followed by the FEATURE_0 ... FEATURE_259 columns

*Prediction accuracy is estimated by macro-f1_score.*


*Public/Private distribution - 50%/50%*

<a name="0."></a><br/>
<font size="4"><b>0. Подгрузка библиотек и самописные функции</b></font>.<br/>

In [27]:
import numpy as np
import pandas as pd

import joblib
import time
import warnings

from plotly.subplots import make_subplots

from sklearn.metrics  import f1_score
from sklearn.metrics  import precision_score
from sklearn.metrics  import recall_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
from termcolor import colored
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns

from scipy import stats as st

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier




from sklearn.model_selection import StratifiedKFold
from sklearn.impute import KNNImputer

from sklearn.metrics import classification_report

In [2]:
#Определяем болд
def bold(): 
    return "\033[1m"

def bold_end(): 
    return "\033[0m"

#Ставим формат для нумериков
pd.options.display.float_format = '{: >10.2f}'.format

#Убираем ворнинги
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#**Function print_basic_info, to display information about the array and its variables.**

#* base - database name
#* info - 1: output information about the array, other: no output
#* describe - 1: output description of array variables, other: no output
#* duplicat - 1: print the number of full duplicates
#* head - n: base example output (output n - lines), n < 1: no output

def print_basic_info(base, info, describe, duplicate, head):
    if info == 1:
        print("\n", bold(), colored('info','green'), bold_end(), "\n")
        print(base.info())
    if head >= 1:
        print("\n", bold(),colored('head','green'),bold_end())
        display(base.head(head))
    if describe == 1:
        print("\n", bold(),colored('describe','green'),bold_end(),"\n")
        for i in base.columns:
            print("\n", bold(), colored(i,'blue'),bold_end(),"\n", base[i].describe())
    if duplicate == 1:
        print("\n", bold(),colored('duplicated','green'),bold_end(),"\n")
        print(base[base.duplicated() == True][base.columns[0]].count())

In [4]:
#__Function ft_namecount__, to display the name of the variable, the frequency normalized table and the description of the variable.

#5 input parameters:

#* *base* - database name
#* *index* - variable name in the database
#* *table* - 1: output frequency normalized table, 0: no output
#* *sort* - 1: sort table by variable labels, 0: no sort
#* *describe* - 1: output variable description, 0: no output

def ft_name_count (base, name , table, sort, describe):
    print(bold(), colored(name,'blue') , bold_end(), "\n")
    if table != 0:
        s = (base[name].value_counts(normalize=True))
        if sort != 0:
            s.sort_index(inplace=True)
        print(s)
    if describe != 0:
        print(base[name].describe())

<a name="1."></a><br/>
<font size="6"><b>1. Подготовка данных</b></font>.<br/>

**Открываем**

In [5]:
#I work locally, online is different

contest_train = pd.read_csv('datasets/contest_train.csv', sep=',',decimal='.' , index_col= 'ID')
contest_test = pd.read_csv('datasets/contest_test.csv', sep=',',decimal='.', index_col = 'ID')

In [6]:
print_basic_info(contest_train,1,0,1,3)


  info  

<class 'pandas.core.frame.DataFrame'>
Index: 24521 entries, 0d1c880d23ff018 to 5044ce165577a9f
Columns: 261 entries, TARGET to FEATURE_259
dtypes: float64(260), int64(1)
memory usage: 49.0+ MB
None

  head 


,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
ID,,,,,,,,,,,,,,,,,,,,,
0d1c880d23ff018,1,2.00,2.00,0.00,0.00,0.00,0.00,0.00,-114.53,-17.22,...,39.00,2.00,223.12,290.02,1.00,1.00,0.00,1.00,2.00,2.00
1579ba37fd82c1f,0,1.00,1.00,0.00,0.00,1.00,0.00,0.00,329.83,NaN,...,25.00,82.00,479.62,611.65,0.00,1.00,0.00,1.00,1.00,1.00
790086f721c7f5e,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.43,-6.68,...,119.00,0.00,352.27,564.57,1.00,1.00,0.00,1.00,1.00,1.00



  duplicated  

0


In [7]:
print_basic_info(contest_test,1,0,1,3)


  info  

<class 'pandas.core.frame.DataFrame'>
Index: 9484 entries, 0749d313171551f to f91280c1f3aeac3
Columns: 260 entries, FEATURE_0 to FEATURE_259
dtypes: float64(260)
memory usage: 18.9+ MB
None

  head 


,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
ID,,,,,,,,,,,,,,,,,,,,,
0749d313171551f,1.00,1.00,0.00,0.00,0.00,0.00,0.00,-46.95,-45.02,0.00,...,0.00,0.00,190.59,162.05,1.00,0.00,1.00,1.00,1.00,1.00
ddb093edbbbe1ef,0.00,0.00,1.00,0.00,0.00,0.00,0.00,343.63,NaN,3.00,...,5.00,109.00,594.57,685.50,1.00,1.00,0.00,1.00,1.00,2.00
cef8538a6054069,0.00,0.00,0.00,0.00,1.00,0.00,0.00,909.18,3260.45,3.00,...,0.00,297.00,10.21,300.40,0.00,0.00,0.00,1.00,1.00,1.00



  duplicated  

0


In [8]:
ft_name_count(contest_train, 'TARGET' , 1, 1, 0)

 TARGET  

0         0.71
1         0.23
2         0.06
Name: TARGET, dtype: float64


The base consists of 24521 cases in the train and 9484 in the <br> 260 features test.

There are no duplicates.

Not all are filled. There are passes.

**TARGET** - There is an imbalance. Strong. The most segment is 0. The least is 2.
Therefore, most likely we will need to boost a part of the sample when training the model.

In [10]:
# Fill in the gaps based on the 4 nearest neighbors.
imputer = KNNImputer(n_neighbors=4)

contest_train_after = imputer.fit_transform(contest_train.drop(['TARGET'], axis=1))
contest_test_aftet = imputer.transform(contest_test)




In [11]:
print(contest_train_after.shape, contest_train.shape, contest_test_aftet.shape)

(24521, 260) (24521, 261) (9484, 260)


Let's make it a Pandas dataframe

In [12]:
pd_contest_train_after = pd.DataFrame(contest_train_after, index = contest_train.index,
                                      columns = contest_train.drop(['TARGET'], axis=1).columns.values)

pd_contest_train_after['TARGET'] = contest_train['TARGET']

pd_contest_test_after = pd.DataFrame(contest_test_aftet, index = contest_test.index ,
                                      columns = contest_test.columns.values)

Check fullness

In [13]:

for i in pd_contest_train_after.columns:
    if pd_contest_train_after[pd_contest_train_after[i].isnull() == True]['TARGET'].sum() > 0:
        print(i, pd_contest_train_after[pd_contest_train_after[i].isnull() == True]['TARGET'].sum())
        
for i in contest_train.columns:
    if contest_train[contest_train[i].isnull() == True]['TARGET'].sum() > 0:
        print(i, contest_train[contest_train[i].isnull() == True]['TARGET'].sum())
        





Clean up from inf values

#### Разабьем данные на выборки

In [14]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

cl_contest_train = clean_dataset(pd_contest_train_after)
cl_contest_test = clean_dataset(pd_contest_test_after)

In [15]:
features = cl_contest_train.drop(['TARGET'], axis=1)

target = contest_train['TARGET']

features_test = cl_contest_test

In [16]:
#break down, stratify
features_train_big, features_valid, target_train_big, target_valid = train_test_split(features, target , test_size=0.20,
                                                                              random_state=515093, stratify = target)

features_train, features_valid = np.array(features_train_big), np.array(features_valid)

#Масштабируем признаки
numeric = features_train_big.columns

scaler = StandardScaler()
scaler.fit(features_train_big[numeric])

display(features_train_big.head())

We take a decision tree as a model and do resampling.

Less than 0 segments, more than 2 segments

In [17]:
def resample(features, target, repeat_down, repeat_up):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    features_two = features[target == 2]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    target_two = target[target == 2]

    features_resampled = pd.concat([features_zeros.sample(n =(int(len(features_zeros)*repeat_down)), 
                                                            replace=False,random_state=2)] + [features_ones] 
                                   + [features_two.sample(n =(int(len(features_two)*repeat_up)), 
                                                            replace=True,random_state=2)])
    
    target_resampled = pd.concat([target_zeros.sample(n =(int(len(target_zeros)*repeat_down)),
                                                        replace=False)] + [target_ones] 
                                 + [target_two.sample(n =(int(len(target_two)*repeat_up)), 
                                                            replace=True)])
    features_resampled, target_resampled = shuffle(features_resampled, target_resampled, random_state=12345)
    features_resampled.reset_index(drop = True, inplace = True)
    target_resampled.reset_index(drop = True, inplace = True)
    return features_resampled, target_resampled

base_up = pd.DataFrame()

model_RFC = RandomForestClassifier(n_estimators = 50, random_state = 2)

for i in np.arange(0.3, 0.5, 0.05):
    for j in np.arange(2.2, 2.7, 0.05):
        features_resampled, target_resampled = resample(features_train_big, target_train_big, i , j)
        model_RFC.fit(features_resampled, target_resampled)
        base_up.loc[str(i) + " " +str(j) ,'Precision'] = precision_score(target_valid, model_RFC.predict(features_valid), average='macro')
        base_up.loc[str(i) + " " +str(j) ,'Recall'] = recall_score(target_valid, model_RFC.predict(features_valid), average='macro')
        base_up.loc[str(i) + " " +str(j),'F1'] = f1_score(target_valid, model_RFC.predict(features_valid), average='macro')

In [19]:
features_resampled, target_resampled = resample(features_train_big, target_train_big, 0.35, 2.5)
features_resampled, target_resampled = resample(features_train_big, target_train_big, 0.39, 2.2)

features_resampled, target_resampled = np.array(features_resampled), target_resampled

In [20]:
target_resampled.value_counts(normalize=True)

0         0.43
1         0.36
2         0.21
Name: TARGET, dtype: float64

<a name="2."></a><br/>
<font size="6"><b>2. Model training </b></font>.<br/>

In [21]:
cv = StratifiedKFold(n_splits=2, random_state=1234, shuffle=True)

In [22]:
#make a function that will record the training time, prediction rate, and prediction quality (RMSE)
def put_in_base_test(base_res, model, features_train, target_train, features_valid, target_valid):
    model.fit(features_train, target_train)
    target = target_valid
    features = features_valid
    
    proba = pd.DataFrame(model.predict_proba(features_valid), columns = ['0','1','2'])

    proba['predict'] = pd.Series(model.predict(features_valid))
    #proba['predict'] = proba['predict'].where(proba['0'] > 0.125, 0)
    
    prediction = test['predict']
    accuracy, precision, recall, f1 = [], [], [], []
    accuracy.append(accuracy_score(target, prediction))
    precision.append(precision_score(target, prediction, average='macro'))
    recall.append(recall_score(target, prediction, average='macro'))
    f1.append(f1_score(target, prediction, average='macro'))
    target_names = ['Seg - 0', 'Seg - 1', 'Seg - 2']
    print(classification_report(target, prediction, target_names=target_names))
    base_res.loc[str(model).split('(')[0],'accuracy'] = np.mean(accuracy)
    base_res.loc[str(model).split('(')[0],'precision'] = np.mean(precision)
    base_res.loc[str(model).split('(')[0],'recall'] = np.mean(recall)
    base_res.loc[str(model).split('(')[0],'f1'] = np.mean(f1)
    
    return base_res, prediction, model

# Display graphs for clarity
def param_bars(base_name, name):
    sns.set(style="whitegrid")
    plt.figure(figsize = (10,3))
    df = base_name
    sns.barplot(data=df, palette="tab20", linewidth=2.5)
    plt.title("Model performance index - " + str(name), fontsize=15)
    plt.ylabel("%%")
    plt.xlabel("Settings")
    plt.ylim((0, 1.2))
    c = 0
    for i in df.columns:
        plt.text( c - 0.1 , df[i].mean() + 0.1, "{0:.0%}".format(df[i].mean()))
        c = c + 1
    plt.show()

In [23]:
features_train, target_train = features_resampled, target_resampled

### AdaBoostClassifier + RandomForestClassifier

* Making a nested model. RandomForestClassifier to AdaBoostClassifier*

* We select the parameters GridSearchCV

In [24]:
#We will select parameters using gridsearch
parameters = {'max_depth':[i for i in range(15,26,1)] , 'n_estimators':[i for i in range(50,500,50)],
              'max_features':[80,90,95,100], 'random_state':[1234]}

clf = GridSearchCV(RandomForestClassifier(), cv = cv, param_grid = parameters, scoring = 'f1_macro')
#clf.fit(features_train, target_train)
#print(clf.best_params_)

In [29]:
grid = {'n_estimators' : [i for i in range(5,25,5)], 'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1.0],
       'algorithm' : ['SAMME', 'SAMME.R']}

clf = AdaBoostClassifier(DecisionTreeClassifier())
rs = GridSearchCV(clf, grid, cv=cv, scoring = 'f1_macro')

#rs.fit(features_train, target_train)
#AdaBoostRegressor_best_params1 = rs.best_params_

check model (very long)

In [ ]:
info_test = pd.DataFrame()

model = AdaBoostClassifier(RandomForestClassifier(max_depth = 20, n_estimators = 500, max_features = 90, random_state = 2),
    n_estimators=200, learning_rate=0.15, algorithm = 'SAMME.R') 

info_test, predictions, final_model = put_in_base_test(info_test, model, features_resampled, target_resampled, features_valid, target_valid)
model = final_model

In [ ]:
param_bars(info_test,'Final Test')

#### export

In [ ]:
output = pd.DataFrame({'ID': contest_test.index, 
                       'Predicted': pd.Series(final_model.predict(features_test), dtype='int32')})

output.to_csv('sub_kiseleva.csv', index=False)
print("Your submission was successfully saved!")

<a name="6."></a><br/>
<font size="6"><b>Output</b></font>.<br/>
[<font size="2">(to content)</font>](#1common.)


* Gaps in columns were filled with KNNImputer - 4
* When training the model, manual rebalancing of the classes was required. 0.39, 2.2,
* The best performance for the model based on AdaBoostClassifier(RandomForestClassifier(max_depth = 19, n_estimators = 500, random_state = 1234), n_estimators=100, learning_rate=0.2)